In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, os, random
import matplotlib.dates as mdates
import pandas as pd
from sklearn.metrics import r2_score
from tqdm import tqdm
from datetime import datetime as DT


def TimeMoveAvg(time, data, window, overlap):
    """
    """
    every, avgtime, avgdata = [], [], []
    t0 = time[0]
    while t0 < time[-1]:
        every.append([t0, t0 + datetime.timedelta(seconds=window)])
        t0 += datetime.timedelta(seconds=overlap)
    
    for j in range(len(every)): 
        d_ = []
        for ti,da in zip(time, data):
            if ti>=every[j][0] and ti<every[j][1]:
                d_.append(da)
        
        if len(d_)>0:
            avgdata.append(sum(d_)/len(d_))
            avgtime.append(every[j][0])
    
    return np.array(avgtime), np.array(avgdata)


def AvgOrSelect(time1, time2, d1, d2):
    """
    """
    # Get start and end time to iterate over
    timezero = time1[0]  if time1[0]  < time2[0]  else time2[0]
    timeneg1 = time1[-1] if time1[-1] > time2[-1] else time2[-1]

    # Create dictionaries for easy access
    DATA1 = { time1[i]:d1[i] for i in range(len(time1)) }
    DATA2 = { time2[i]:d2[i] for i in range(len(time2)) }

    # print(int((timeneg1-timezero).total_seconds() + 100))
    timesteps = [ timezero + datetime.timedelta(seconds=j) for j in range(int((timeneg1-timezero).total_seconds() + 100)) ]
    OUTPUT_TEMP = { timesteps[i]:None for i in range(len(timesteps)) }

    for n in range(len(timesteps)):

        # Case where we averaged the two
        if timesteps[n] in DATA1.keys():
            if timesteps[n] in DATA2.keys():
                OUTPUT_TEMP[timesteps[n]] = (DATA1[timesteps[n]] + DATA2[timesteps[n]])/2.0
            
            # Only in Data 1
            else:
                OUTPUT_TEMP[timesteps[n]] = DATA1[timesteps[n]]
        
        # Only in Data 2
        elif timesteps[n] in DATA2.keys():
            OUTPUT_TEMP[timesteps[n]] = DATA2[timesteps[n]]

    OUTPUT = dict()
    for k,v in OUTPUT_TEMP.items():
        if v:
            OUTPUT[k] = v
    ST_  = sorted(OUTPUT.items())

    return [j[0] for j in ST_], [j[1] for j in ST_]


def MakeDataFromText(filepath:str)->dict:
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif line.count(":")==2:
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s)))
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db


def MakeDict(data:list):
    """
    """
    t1234 = dict()
    for d in data:
        for k,v in d.items():
            if k in t1234: # if tag is already in dict
                for i in range(len(v[0])):
                    if v[0][i] in t1234[k]:
                        t1234[k][v[0][i]].append(v[1][i])
                    else:
                        t1234[k][v[0][i]] = [v[1][i]]
            else:             # if tag is not in dict
                t1234[k] = dict()
                for i in range(len(v[0])):
                    t1234[k][v[0][i]] = [v[1][i]]
    for k,v in t1234.items():
        for k2,v2 in v.items():
            t1234[k][k2] = sum(v2)/len(v2)
    return t1234


def Chop(time, data, delin1, delin2):
    """
    Chop a set of lists based on the time frame given.
    """
    new_time = [ time[j] for j in range(len(time)) if time[j] > delin1 and time[j] < delin2 ]
    new_data = [ data[j] for j in range(len(time)) if time[j] > delin1 and time[j] < delin2 ]
    return new_time, new_data

- test #1 s2a s3a -> 2:20.15
- test #1 s0a s0a -> 2:23.45
- test #1 s2b s3a -> 2:25.60
- test #1 s1a s2b -> 2:25.81

- duty5s -> 2:29.00
- single s0a -> 2:28.10


s0a s0a -> 507, 120 \
s2a s3a -> 480, 112 \
s2b s3a -> 401, 106 \
s1a s2b -> 493, 116 

duty 5s (s0a) -> 209, 44 

single (s0a) -> 543, 75

In [ ]:
# t1_s0s0 = MakeDataFromText("Lab_Testing/Test1/tags_1_s0a_s0a.txt"); t1_s0s0 = MakeDict( [t1_s0s0] )
# t1_s2s3 = MakeDataFromText("Lab_Testing/Test1/tags_1_s2a_s3a.txt"); t1_s2s3 = MakeDict( [t1_s2s3] )
# t_s2bs3a = MakeDataFromText("Lab_Testing/Test1/tags_1_s2b_s3a.txt"); t_s2bs3a = MakeDict( [t_s2bs3a] )
# t_s1as2b = MakeDataFromText("Lab_Testing/Test1/tags_1_s1a_s2b.txt"); t_s1as2b = MakeDict( [t_s1as2b] )
# t_duty5s = MakeDataFromText("Lab_Testing/Test1/tags_1_duty5.txt"); t_duty5s = MakeDict( [t_duty5s] )
# t_single = MakeDataFromText("Lab_Testing/Test1/tags_1_single.txt"); t_single = MakeDict( [t_single] )

# t2_both = MakeDataFromText("Lab_Testing/Test2/tags_both.txt"); t2_both = MakeDict( [t2_both] )
# t2_left = MakeDataFromText("Lab_Testing/Test2/tags_left.txt"); t2_left = MakeDict( [t2_left] )
# t2_right = MakeDataFromText("Lab_Testing/Test2/tags_right.txt"); t2_right = MakeDict( [t2_right] )
# t2_both2 = MakeDataFromText("Lab_Testing/Test2/tags_both2.txt"); t2_both2 = MakeDict( [t2_both2] )
# r2_both = MakeDataFromText("Lab_Testing/Test2/rssi_both.txt"); r2_both = MakeDict( [r2_both] )
# r2_left = MakeDataFromText("Lab_Testing/Test2/rssi_left.txt"); r2_left = MakeDict( [r2_left] )
# r2_right = MakeDataFromText("Lab_Testing/Test2/rssi_right.txt"); r2_right = MakeDict( [r2_right] )
# r2_both2 = MakeDataFromText("Lab_Testing/Test2/rssi_both2.txt"); r2_both2 = MakeDict( [r2_both2] )


t_close1 = MakeDataFromText("Lab_Testing/Test3/tags_close1.txt"); t_close1 = MakeDict( [t_close1] )
r_close1 = MakeDataFromText("Lab_Testing/Test3/rssi_close1.txt"); r_close1 = MakeDict( [r_close1] )
t_far1 = MakeDataFromText("Lab_Testing/Test3/tags_far1.txt"); t_far1 = MakeDict( [t_far1] )
r_far1 = MakeDataFromText("Lab_Testing/Test3/rssi_far1.txt"); r_far1 = MakeDict( [r_far1] )

t_bothclose1 = MakeDataFromText("Lab_Testing/Test3/tags_bothclose1.txt"); t_bothclose1 = MakeDict( [t_bothclose1] )
r_bothclose1 = MakeDataFromText("Lab_Testing/Test3/rssi_bothclose1.txt"); r_bothclose1 = MakeDict( [r_bothclose1] )
t_bothfar1 = MakeDataFromText("Lab_Testing/Test3/tags_bothfar1.txt"); t_bothfar1 = MakeDict( [t_bothfar1] )
r_bothfar1 = MakeDataFromText("Lab_Testing/Test3/rssi_bothfar1.txt"); r_bothfar1 = MakeDict( [r_bothfar1] )

t_close2 = MakeDataFromText("Lab_Testing/Test3/tags_close2.txt"); t_close2 = MakeDict( [t_close2] )
r_close2 = MakeDataFromText("Lab_Testing/Test3/rssi_close2.txt"); r_close2 = MakeDict( [r_close2] )
t_far2 = MakeDataFromText("Lab_Testing/Test3/tags_far2.txt"); t_far2 = MakeDict( [t_far2] )
r_far2 = MakeDataFromText("Lab_Testing/Test3/rssi_far2.txt"); r_far2 = MakeDict( [r_far2] )

In [ ]:
# for d_ in [t_single, t_duty5s, t1_s0s0, t_s2bs3a, t_s1as2b]:
#     total_rssi = []
#     for k,v in d_.items():
#         r = sorted(v.items())
#         total_rssi += [ j[1] for j in r ]
#     print(np.mean(total_rssi))

reads = 0
for k,v in t_close1.items():
    reads += len(v)
print(reads, len(t_close1.keys()) )

reads = 0
for k,v in t_close2.items():
    reads += len(v)
print(reads, len(t_close2.keys()) )

reads = 0
for k,v in t_far1.items():
    reads += len(v)
print(reads, len(t_far1.keys()) )

reads = 0
for k,v in t_far2.items():
    reads += len(v)
print(reads, len(t_far2.keys()) )



# reads = 0
# for k,v in t_bothclose1.items():
#     reads += len(v)
# print(reads, len(t_bothclose1.keys()) )
# reads = 0
# for k,v in t_bothfar1.items():
#     reads += len(v)
# print(reads, len(t_bothfar1.keys()) )



total_rssi = []
for k,v in r_far1.items():
    r = sorted(v.items())
    total_rssi += [ j[1] for j in r ]
print(np.mean(total_rssi))

total_rssi = []
for k,v in r_far2.items():
    r = sorted(v.items())
    total_rssi += [ j[1] for j in r ]
print(np.mean(total_rssi))

In [ ]:
# total_rssi = []
# for k,v in r2_both.items():
#     r = sorted(v.items())
#     total_rssi += [ j[1] for j in r ]
# print(np.mean(total_rssi))

# total_rssi = []
# for k,v in r2_both2.items():
#     r = sorted(v.items())
#     total_rssi += [ j[1] for j in r ]
# print(np.mean(total_rssi))

# total_rssi = []
# for k,v in r2_left.items():
#     r = sorted(v.items())
#     total_rssi += [ j[1] for j in r ]
# print(np.mean(total_rssi))

# total_rssi = []
# for k,v in r2_right.items():
#     r = sorted(v.items())
#     total_rssi += [ j[1] for j in r ]
# print(np.mean(total_rssi))

# for d_ in [t_single, t_duty5s, t1_s0s0, t_s2bs3a, t_s1as2b]:
#     total_rssi = []
#     for k,v in d_.items():
#         r = sorted(v.items())
#         total_rssi += [ j[1] for j in r ]
#     print(np.mean(total_rssi))

In [ ]:
# reads = 0
# for k,v in t2_both.items():
#     reads += len(v)
# print(reads, len(t2_both.keys()) )

# reads = 0
# for k,v in t2_both2.items():
#     reads += len(v)
# print(reads, len(t2_both2.keys()) )

# reads = 0
# for k,v in t2_left.items():
#     reads += len(v)
# print(reads, len(t2_left.keys()) )

# reads = 0
# for k,v in t2_right.items():
#     reads += len(v)
# print(reads, len(t2_right.keys()) )